In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import typing

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import defaultdict

/usr/local/anaconda3/envs/pitch-sequencing/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [3]:
from pitch_sequencing.ml.tokenizers.pitch_sequence import PitchSequenceWithCountTokenizer

tokenizer = PitchSequenceWithCountTokenizer()
tokenizer.tokenize("FF,FF", "0-0,0-1,1-1")

([1,
  13,
  8,
  14,
  8,
  17,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [False,
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True])

In [9]:
from pitch_sequencing.ml.models.last_pitch import LastPitchTransformerModel
import pitch_sequencing.ml.data.count_state as cs 
from dataclasses import asdict
model = LastPitchTransformerModel(tokenizer.vocab_size(), 64, 4, 2)
input, mask = tokenizer.tokenize("FF,FF", "0-0,0-1,1-1")
input = torch.tensor([input])
mask = torch.tensor([mask])
#input = torch.Tensor(input).to(torch.long).unsqueeze(0)
#mask = torch.Tensor(mask).to(torch.bool).unsqueeze(0)
print(input)
print(mask)
print(input.size())
print(mask.size())

input_data = cs.SingularSequence(input, mask)

model(**asdict(input_data))

tensor([[ 1, 13,  8, 14,  8, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([[False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True]])
torch.Size([1, 64])
torch.Size([1, 64])
tensor([[ 1, 13,  8, 14,  8, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

tensor([[-0.6189,  0.1143,  0.0244,  0.5220,  1.1010,  0.1627,  0.1047,  0.3541,
          0.1873, -0.8326,  0.2494,  0.3838,  0.5323, -0.0671,  0.3744, -0.8140,
          0.9306, -0.1773, -0.4971, -0.4289, -0.0389, -0.2302,  0.8830, -0.4407,
         -0.0978]], grad_fn=<AddmmBackward0>)

In [10]:
exploded_test_df = pd.read_csv('gs://pitch-sequencing/sequence_data/full_sequence_data/exploded/large_cur_test.csv')

In [22]:
test_dataset = cs.LastPitchSequenceWithCountDataset(exploded_test_df, tokenizer, expand_target=False, seq_df_key="pitch_sequence")
test_dataloader = DataLoader(test_dataset, batch_size=4, collate_fn=cs.collate_interleaved_and_target)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
import gcsfs 

fs = gcsfs.GCSFileSystem()
model_path = "gs://pitch-sequencing/training_runs/countstate_training_job_20241014095006_attn_masking_cel/final/model.pth"
interleaved_trained_model = LastPitchTransformerModel(tokenizer.vocab_size(), d_model=64, nhead=4, num_layers=2)
with fs.open(model_path, "rb") as f:
   interleaved_trained_model.load_state_dict(torch.load(f, map_location=torch.device('cpu') ))

TypeError: super(type, obj): obj must be an instance or subtype of type

In [24]:
for batch in test_dataloader:
    input_data, target = [b.to(device) for b in batch]
    output = model(**asdict(input_data)) 
    print(output)
    break


tensor([[-0.7033,  0.0179,  0.3454,  0.8255,  1.4692,  0.2941,  0.1398,  0.1461,
          0.3340, -1.2284,  0.3960,  0.3626,  0.6437, -0.0827,  0.4881, -0.3058,
          0.7603,  0.1965, -0.5134, -0.0382, -0.3277,  0.0252,  0.8822, -0.8045,
         -0.0230],
        [-0.5018, -0.2100,  0.0402,  0.8873,  1.3357,  0.1921, -0.0615,  0.1461,
          0.1339, -1.2506,  0.4494,  0.6110,  0.2990,  0.0741,  0.6917, -0.4307,
          0.5992,  0.1254, -0.5720, -0.1667, -0.2134,  0.0377,  0.7336, -0.9893,
          0.0042],
        [-0.5778, -0.0820,  0.0711,  0.7204,  1.3623,  0.1380,  0.1401,  0.2097,
          0.4762, -1.1766,  0.2308,  0.5696,  0.5547,  0.0717,  0.5384, -0.3625,
          0.6421,  0.1388, -0.5663, -0.1888, -0.2466,  0.1071,  0.7160, -0.8247,
          0.1857],
        [-0.9128,  0.1367, -0.0835,  0.4694,  1.3833,  0.2088, -0.0193,  0.4249,
          0.0610, -0.9928,  0.2075,  0.2810,  0.4219,  0.0426,  0.5210, -0.5045,
          0.9126,  0.0750, -0.5971, -0.4536, -0.2918